In [1]:
import pandas as pd, spacy, nltk, numpy as np, re, ssl

In [2]:
df = pd.read_excel("DataPanelWHR2021C2.xls", engine="xlrd")

In [3]:
df.head()

,Country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.723590,7.370100,0.450662,50.799999,0.718114,0.167640,0.881686,0.517637,0.258195
1,Afghanistan,2009,4.401778,7.539972,0.552308,51.200001,0.678896,0.190099,0.850035,0.583926,0.237092
2,Afghanistan,2010,4.758381,7.646709,0.539075,51.599998,0.600127,0.120590,0.706766,0.618265,0.275324
3,Afghanistan,2011,3.831719,7.619532,0.521104,51.919998,0.495901,0.162427,0.731109,0.611387,0.267175
4,Afghanistan,2012,3.782938,7.705479,0.520637,52.240002,0.530935,0.236032,0.775620,0.710385,0.267919


In [4]:
df_sorted = df.sort_values(by=['year'], ascending = False)

In [5]:
df_sorted.head()

,Country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
1948,Zimbabwe,2020,3.159802,7.828757,0.717243,56.799999,0.643303,-0.008696,0.788523,0.702573,0.345736
174,Benin,2020,4.407746,8.102292,0.506636,55.099998,0.783115,-0.083489,0.531884,0.608585,0.304512
1835,United Kingdom,2020,6.798177,10.625811,0.929353,72.699997,0.884624,0.202508,0.490204,0.758164,0.224655
1394,Philippines,2020,5.079585,9.061443,0.781140,62.099998,0.932042,-0.115543,0.744284,0.803562,0.326889
785,Iraq,2020,4.785165,9.167186,0.707847,61.400002,0.700215,-0.020748,0.849109,0.644464,0.531539


In [6]:
df_dedup = df_sorted.drop_duplicates(subset=['Country'])

In [7]:
df_dedup.head()

,Country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
1948,Zimbabwe,2020,3.159802,7.828757,0.717243,56.799999,0.643303,-0.008696,0.788523,0.702573,0.345736
174,Benin,2020,4.407746,8.102292,0.506636,55.099998,0.783115,-0.083489,0.531884,0.608585,0.304512
1835,United Kingdom,2020,6.798177,10.625811,0.929353,72.699997,0.884624,0.202508,0.490204,0.758164,0.224655
1394,Philippines,2020,5.079585,9.061443,0.781140,62.099998,0.932042,-0.115543,0.744284,0.803562,0.326889
785,Iraq,2020,4.785165,9.167186,0.707847,61.400002,0.700215,-0.020748,0.849109,0.644464,0.531539


In [8]:
len(df_sorted)

1949

In [9]:
len(df_dedup)

166

In [10]:
df_csv = pd.read_csv("countries.csv")
df_csv.head()

,Country,Continent
0,Algeria,Africa
1,Angola,Africa
2,Benin,Africa
3,Botswana,Africa
4,Burkina,Africa


In [11]:
len(df_csv)

194

In [12]:
c1 = df_dedup["Country"]
c2 = list(df_csv["Country"])
c3 = [(country, country in c2) for country in c1]

In [13]:
num = 0
missing = []
for pair in c3:
    if pair[1]:
        num +=1
    else:
        missing.append(pair[0])    
num
missing

['Hong Kong S.A.R. of China',
 'Russia',
 'North Macedonia',
 'Kosovo',
 'United States',
 'Czech Republic',
 'South Korea',
 'Taiwan Province of China',
 'Myanmar',
 'Burkina Faso',
 'Palestinian Territories',
 'North Cyprus',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Somaliland region']

In [14]:
df_csv.loc[df_csv['Country'] == "Madagascar", 'Continent'].iloc[0]

'Africa'

In [15]:
df_dedup.head()

,Country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
1948,Zimbabwe,2020,3.159802,7.828757,0.717243,56.799999,0.643303,-0.008696,0.788523,0.702573,0.345736
174,Benin,2020,4.407746,8.102292,0.506636,55.099998,0.783115,-0.083489,0.531884,0.608585,0.304512
1835,United Kingdom,2020,6.798177,10.625811,0.929353,72.699997,0.884624,0.202508,0.490204,0.758164,0.224655
1394,Philippines,2020,5.079585,9.061443,0.781140,62.099998,0.932042,-0.115543,0.744284,0.803562,0.326889
785,Iraq,2020,4.785165,9.167186,0.707847,61.400002,0.700215,-0.020748,0.849109,0.644464,0.531539


In [16]:
df_cont = df_dedup.set_index('Country').join(df_csv.set_index('Country'), on='Country', how='left')

In [17]:
df_cont = df_cont.reset_index()

In [18]:
len(df_cont)

166

In [19]:
df_cont.head()

,Country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Continent
0,Zimbabwe,2020,3.159802,7.828757,0.717243,56.799999,0.643303,-0.008696,0.788523,0.702573,0.345736,Africa
1,Benin,2020,4.407746,8.102292,0.506636,55.099998,0.783115,-0.083489,0.531884,0.608585,0.304512,Africa
2,United Kingdom,2020,6.798177,10.625811,0.929353,72.699997,0.884624,0.202508,0.490204,0.758164,0.224655,Europe
3,Philippines,2020,5.079585,9.061443,0.781140,62.099998,0.932042,-0.115543,0.744284,0.803562,0.326889,Asia
4,Iraq,2020,4.785165,9.167186,0.707847,61.400002,0.700215,-0.020748,0.849109,0.644464,0.531539,Asia


In [20]:
# I updated the name of the output so that I don't accidentally overwrite the manual work I did at the end to add in the last few outliers.
#df_cont.to_csv("Assets/Countries/base-combined-countries.csv")

In [21]:
df3 = df_csv.set_index('Country').join(df_dedup.set_index('Country'), on='Country', how='left')

In [22]:
df3 = df3[df3.year.isnull()]

In [23]:
df3

,Continent,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
Country,,,,,,,,,,,
Burkina,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cape Verde,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Congo,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Congo, Democratic Republic of",Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Equatorial Guinea,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Eritrea,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Guinea-Bissau,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sao Tome and Principe,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seychelles,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> Note that I updated these in the spreadsheet manually with Excel because it was faster to do it by hand... I should go back when I have time to do it programmatically...

In [26]:
df_final = pd.read_csv('combined-countries.csv')

In [27]:
df_final.head()

,Words,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Categories
0,Afghanistan,2019.0,2.375092,7.697248,0.419973,52.400002,0.393656,-0.108459,0.923849,0.351387,0.502474,Asia
1,Albania,2020.0,5.364910,9.497252,0.710115,69.300003,0.753671,0.006968,0.891359,0.678661,0.265066,Europe
2,Algeria,2019.0,4.744627,9.336946,0.803259,66.099998,0.385083,0.005087,0.740609,0.584944,0.215198,Africa
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe
4,Angola,2014.0,3.794838,9.016735,0.754615,54.599998,0.374542,-0.167723,0.834076,0.578517,0.367864,Africa


In [29]:
words = df_final.Words.apply(lambda x: x.lower())

In [30]:
words

0      afghanistan
1          albania
2          algeria
3          andorra
4           angola
          ...     
192      venezuela
193        vietnam
194          yemen
195         zambia
196       zimbabwe
Name: Words, Length: 197, dtype: object